# 1. Import Package

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sklearn
import time
import numpy as np
import json
import pandas as pd
import random
# import pdfminer
# import pdfminer.high_level
import os
pd.set_option("max_colwidth", 400)
from pandarallel import pandarallel

 # 2. get all the urls of articles from the search 

In [2]:
base_url =  'https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date'
df = pd.DataFrame([base_url + '&p=%s'%i for i in range(1,3336+1)],  columns =['url'] )
df

,url
0,https://www.amnesty.org/en/search/?q=&ref=&yea...
1,https://www.amnesty.org/en/search/?q=&ref=&yea...
2,https://www.amnesty.org/en/search/?q=&ref=&yea...
3,https://www.amnesty.org/en/search/?q=&ref=&yea...
4,https://www.amnesty.org/en/search/?q=&ref=&yea...
...,...
3331,https://www.amnesty.org/en/search/?q=&ref=&yea...
3332,https://www.amnesty.org/en/search/?q=&ref=&yea...
3333,https://www.amnesty.org/en/search/?q=&ref=&yea...
3334,https://www.amnesty.org/en/search/?q=&ref=&yea...


In [3]:
# def extract_urls(url):
#     coverpage = requests.get(url).content
#     soup = BeautifulSoup(coverpage, 'html.parser')
#     return soup.find_all('article')

def extract_urls(url):
    coverpage = requests.get(url).content
    soup = BeautifulSoup(coverpage, 'html.parser')
    return [str(i) for i in soup.find_all('article')]


In [4]:
pandarallel.initialize(nb_workers=512)
df[['articles']] = df['url'].parallel_apply(extract_urls)

INFO: Pandarallel will run on 512 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
df = df.explode('articles').reset_index(drop=True)
df

,url,articles
0,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
1,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
2,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
3,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
4,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
...,...,...
48121,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
48122,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
48123,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
48124,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."


In [7]:
# df.to_csv('amnesty_raw_articles.csv',index= False)

# 3. Extract html per articles

In [2]:
df = pd.read_csv('amnesty_raw_articles.csv',header = [0])
df

,url,articles
0,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
1,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
2,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
3,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
4,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
...,...,...
53364,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
53365,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
53366,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
53367,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."


In [3]:
df1= df.copy()
df1

,url,articles
0,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
1,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
2,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
3,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
4,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
...,...,...
53364,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
53365,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
53366,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."
53367,https://www.amnesty.org/en/search/?q=&ref=&yea...,"<article class=""search-item--list"">\n<a class=..."


In [14]:
def extract_rows(article):
    article = BeautifulSoup(article, 'html.parser')
    content_type = re.sub("^\s+|\s+$", "", article.span.text, flags=re.UNICODE)        
    headline = re.sub("^\s+|\s+$", "", article.h1.text, flags=re.UNICODE)
    abstract = re.sub("^\s+|\s+$", "", article.p.text, flags=re.UNICODE) if article.p else ''
    date = re.sub("^\s+|\s+$", "", article.dd.text, flags=re.UNICODE) if article.dd else ''
    
    if not article.a.has_attr('href'):
        link=''
    else:
        link = article.a['href']
        
        if link.startswith('/'):
            link = 'https://www.amnesty.org' + link
        elif len(link)<5:
            link = ''
    return pd.Series([headline, abstract, date, content_type, link])


headers={
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36",
}

def get_html(link):
    time.sleep(random.uniform(2,10))
    
    try:
        link_page = requests.get(link, headers=headers).content
    except:
        print('exception', link)
        return 
        
    link_soup = BeautifulSoup(link_page, 'html.parser')
    find_content = link_soup.find_all('div', class_ = re.compile('col__content'))
    if len(find_content):
        return str(find_content[0])
    else:
        print('No content', link)
        return 
    
    

def get_read_more(url):
    print(url)

    if url =='':
        return
    
    try:
        read_more_page = requests.get(url).content      
        read_more_soup = BeautifulSoup(read_more_page, 'html.parser')
        read_more_find = read_more_soup.find_all('div', class_='wysiwyg')
        read_more_text = read_more_find[0].text if len(read_more_find)>0 else ''
        read_more_text =  re.sub("^\s+|\s+$", "", full_story, flags=re.UNICODE) 
        return read_more_text
        
    except:
        print('exception', url)
        return

In [3]:
pandarallel.initialize(nb_workers=36)

INFO: Pandarallel will run on 36 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
df1[['headline', 'abstract', 'date', 'content_type', 'link']] =\
df1['articles'].parallel_apply(extract_rows).values

In [32]:
batch_size = 512
for i in range(28132,df1.shape[0],batch_size):
    print(i ,end = '\r')
    df1.loc[i:i+batch_size-1,'html'] = df1.loc[i:i+batch_size-1,'link'].parallel_apply(get_html).values

No content https://www.amnesty.org/en/documents/mde13/088/2008/en/
No content https://www.amnesty.org/en/documents/mde13/052/2008/en/
No content https://www.amnesty.org/en/documents/amr41/060/2007/en/
No content https://www.amnesty.org/en/documents/amr51/130/2007/en/
No content https://www.amnesty.org/en/documents/act76/003/2007/en/
No content https://www.amnesty.org/en/documents/mde13/035/2007/en/
No content https://www.amnesty.org/en/documents/asa33/050/2006/en/
No content https://www.amnesty.org/en/documents/amr37/007/2006/en/
No content https://www.amnesty.org/en/documents/asa39/011/2006/en/
No content https://www.amnesty.org/en/documents/afr25/026/2006/en/
No content https://www.amnesty.org/en/documents/eur46/027/2006/en/
No content https://www.amnesty.org/en/documents/mde14/028/2006/en/
No content https://www.amnesty.org/en/documents/afr54/011/2006/en/
No content https://www.amnesty.org/en/documents/amr51/170/2005/en/
No content https://www.amnesty.org/en/documents/mde20/005/2005

In [21]:
df1.to_csv('amnesty_html.csv',index= False)

In [16]:
df1 = df1.loc[~df1.html.isnull()]

In [13]:
# df1.loc[df1.html=='']
df1.loc[df1.html.isnull(),'html'] = df1.loc[df1.html.isnull(),'link'].parallel_apply(get_html).values

exception nan
exception nan
exception nan
No content https://www.amnesty.org/en/latest/campaigns/2015/06/stop-torture-26-june/
No content https://www.amnesty.org/en/latest/campaigns/2015/06/does-your-country-share-your-data-with-the-usa/
No content https://www.amnesty.org/en/latest/research/2015/02/death-sentences-and-executions-2014/
No content https://www.amnesty.org/en/latest/news/2015/01/postponement-raif-badawi-flogging-medical-grounds-exposes-shocking-brutality-punishment-2015-01/
No content https://www.amnesty.org/en/latest/news/2014/01/cambodia-police-kill-striking-garment-workers-2014-01-03/
No content https://www.amnesty.org/en/latest/news/2013/08/syria-scm-trial-2013-08-20/
No content https://www.amnesty.org/en/latest/news/2013/06/libya-must-drop-charges-against-politicians-over-women-s-rights-cartoon-2013-06-14/
No content https://www.amnesty.org/en/latest/news/2013/05/israel-must-remove-new-settler-outpost-west-bank-2013-05-02/
No content https://www.amnesty.org/en/latest/

In [ ]:
df1.to_csv('amnesty_html.csv',index= False)

 # 4. Extract headlines, abstract and read more per html

In [2]:
df1= pd.read_csv('amnesty_html.csv',header= [0])

In [3]:
df2 = df1.copy()
df2

,url,articles,headline,abstract,date,content_type,link,html,full_story,topics,read_more_link,pdf_link,read_more_content,pdf_content
0,https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date&p=1,"<article class=""search-item--list"">\n<a class=""search-item__link search-item__link--list clearfix"" data-search-link="""" href=""/en/latest/research/2021/02/vietnam-phishing-attacks-research-vn/"">\n<d...",Nhấp chuột và Mồi nhử: Các nhà bảo vệ nhân quyền Việt Nam bị tấn công bằng phần mềm gián điệp,Một cuộc điều tra mới của Ân xá Quốc tế vừa xác định một chiến dịch tấn công bằng phần mềm gián điệp nhắm vào những người bảo vệ nhân quyền (HRDs) Việt Nam từ tháng 2 năm 2018 đến tháng 11 năm 202...,24 February 2021,Blog,https://www.amnesty.org/en/latest/research/2021/02/vietnam-phishing-attacks-research-vn/,"<div class=""col__content"">\n<h1 class=""h2 heading--in-padded"" data-start-content="""">Nhấp chuột và Mồi nhử: Các nhà bảo vệ nhân quyền Việt Nam bị tấn công bằng phần mềm gián điệp </h1>\n<div class=...",Một cuộc điều tra mới của Ân xá Quốc tế vừa xác định một chiến dịch tấn công bằng phần mềm gián điệp nhắm vào những người bảo vệ nhân quyền (HRDs) Việt Nam từ tháng 2 năm 2018 đến tháng 11 năm 202...,[],NaN,NaN,NaN,NaN
1,https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date&p=1,"<article class=""search-item--list"">\n<a class=""search-item__link search-item__link--list clearfix"" data-search-link="""" href=""/en/latest/research/2021/02/vietnam-hacking-group-activists-vn-pr/"">\n<...",Các nhà hoạt động Việt Nam bị nhóm hacker khét tiếng nhắm mục tiêu,"Nhóm chuyên xâm nhập mạng Ocean Lotus, nhóm bị nghi ngờ có liên hệ với chính phủ Việt Nam, là tổ chức đứng đằng sau một chiến dịch tấn công kéo dài dùng phần mềm gián điệp nhắm vào các nhà hoạt độ...",24 February 2021,Article,https://www.amnesty.org/en/latest/research/2021/02/vietnam-hacking-group-activists-vn-pr/,"<div class=""col__content"">\n<h1 class=""heading--main heading--in-padded"" data-start-content="""">\r\n\t\tCác nhà hoạt động Việt Nam bị nhóm hacker khét tiếng nhắm mục tiêu \r\n\t</h1>\n<div class=""m...","Nhóm chuyên xâm nhập mạng Ocean Lotus, nhóm bị nghi ngờ có liên hệ với chính phủ Việt Nam, là tổ chức đứng đằng sau một chiến dịch tấn công kéo dài dùng phần mềm gián điệp nhắm vào các nhà hoạt độ...",[],NaN,NaN,NaN,NaN
2,https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date&p=1,"<article class=""search-item--list"">\n<a class=""search-item__link search-item__link--list clearfix"" data-search-link="""" href=""/en/latest/news/2021/02/viet-nam-hacking-group-targets-activist/"">\n<di...",Vietnamese activists targeted by notorious hacking group,"Hacking group Ocean Lotus, which has been suspected of having links with the Vietnamese government, is behind a sustained campaign of spyware attacks on the country’s human rights activists, a new...",24 February 2021,News,https://www.amnesty.org/en/latest/news/2021/02/viet-nam-hacking-group-targets-activist/,"<div class=""col__content"">\n<h1 class=""heading--main heading--in-padded"" data-start-content="""">\r\n\t\tVietnamese activists targeted by notorious hacking group\r\n\t</h1>\n<div class=""meta"">\n<div...",NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date&p=1,"<article class=""search-item--list"">\n<a class=""search-item__link search-item__link--list clearfix"" data-search-link="""" href=""/en/latest/news/2021/02/pakistan-christian-couple-death-row/"">\n<div cl...",Pakistan: Christian couple on death row for ‘blasphemous texts’ must be released,"The Pakistani authorities must immediately and unconditionally release a husband and wife facing death sentences after being convicted of sending ‘blasphemous’ text messages, said Amnesty Internat...",24 February 2021,Article,https://www.amnesty.org/en/latest/news/2021/02/pakistan-christian-couple-death-row/,"<div class=""col__content"">\n<h1 class=""heading--main heading--in-pa

In [4]:
def extract_html(html):
    
    soup = BeautifulSoup(html, 'html.parser')

    # Content #

    full_story_find = soup.find_all('div', class_='wysiwyg')
    if len(full_story_find):
        full_story = full_story_find[0].text
        full_story =  re.sub("^\s+|\s+$", "", full_story, flags=re.UNICODE) 
    else:
        full_story = ''

    # Topic #    

    link_topics = soup.find_all('a',class_ ='tags__link')
    if len(link_topics):
        topics = [re.sub("^\s+|\s+$", "", i.text, flags=re.UNICODE) for i in link_topics]
    else:
        topics=''    

        
    # Read More #

    read_more_find = soup.find_all('a', string = 'Read more')

    if len(read_more_find):
        read_more_link = read_more_find[0]['href']
        if read_more_link[0] == '/':
            read_more_link = 'https://www.amnesty.org' + read_more_link
    else:
        read_more_link = ''    
        
    # report #    
    
    pdf_find =[]
  
    if soup.find_all(string= re.compile('Choose a language to view report')):
        pdf_find = soup.find_all('a', {'href': re.compile('/download/.*')}, string = re.compile('English'))
    elif soup.find_all(string= re.compile('View report in English')):
        pdf_find = soup.find_all('a', {'href': re.compile('/download/.*')}, string = re.compile('Download PDF'))

    pdf_link = 'https://www.amnesty.org'+ pdf_find[0]['href'] if len(pdf_find) else ''
    
    return pd.Series([full_story, topics, read_more_link, pdf_link])

In [5]:
pandarallel.initialize(nb_workers=128)
df2[['full_story', 'topics', 'read_more_link', 'pdf_link']] = df2['html'].parallel_apply(extract_html).values

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
df2.loc[df2.read_more_link!='']

,url,articles,headline,abstract,date,content_type,link,html,full_story,topics,read_more_link,pdf_link,read_more_content,pdf_content
142,https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date&p=9,"<article class=""search-item--list"">\n<a class=""search-item__link search-item__link--list clearfix"" data-search-link="""" href=""/en/latest/campaigns/2021/02/prisoners-of-conscience-in-the-gulf/"">\n<d...",Silence is King: Prisoners of conscience in the gulf,"The 2011 so-called “Arab Spring” has turned into a long winter for advocates of change – political and human rights activists, including prisoners of conscience. Scores who peacefully expressed th...",2 February 2021,Blog,https://www.amnesty.org/en/latest/campaigns/2021/02/prisoners-of-conscience-in-the-gulf/,"<div class=""col__content"">\n<h1 class=""h2 heading--in-padded"" data-start-content="""">Silence is King: Prisoners of conscience in the gulf</h1>\n<div class=""meta"">\n<div class=""meta__section"">\n<div...","The 2011 so-called “Arab Spring” has turned into a long winter for advocates of change – political and human rights activists, including prisoners of conscience. Scores who peacefully expressed th...","[Bahrain, Oman, Saudi Arabia, United Arab Emirates, Human Rights Defenders and Activists]",https://www.amnesty.org/en/latest/news/2017/09/saudi-arabia-wave-of-arrests-targets-last-vestiges-of-freedom-of-expression/,,NaN,NaN
8337,https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date&p=522,"<article class=""search-item--list"">\n<a class=""search-item__link search-item__link--list clearfix"" data-search-link="""" href=""/en/latest/campaigns/2017/05/its-not-marriage-but-equality-that-we-long...",It's not marriage but equality that we long for,"On this island of Taiwan, there have always been people whose rights are not protected. Overtime, we start to believe that everyone is treated equally but reality has a way of awakening us from th...",2 May 2017,Blog,https://www.amnesty.org/en/latest/campaigns/2017/05/its-not-marriage-but-equality-that-we-long-for/,"<div class=""col__content"">\n<h1 class=""h2 heading--in-padded"" data-start-content="""">It's not marriage but equality that we long for</h1>\n<div class=""meta"">\n<div class=""meta__section"">\r\nBy ...","On this island of Taiwan, there have always been people whose rights are not protected. Overtime, we start to believe that everyone is treated equally but reality has a way of awakening us from th...","[Asia and The Pacific, Taiwan, LGBTI Rights]",https://www.amnesty.org/en/documents/asa38/5935/2017/en/,,NaN,NaN
13309,https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date&p=833,"<article class=""search-item--list"">\n<a class=""search-item__link search-item__link--list clearfix"" data-search-link="""" href=""/en/latest/news/2015/03/buddhist-nationalists-stoke-hatred-in-myanmar/""...",Buddhist nationalists stoke hatred in Myanmar,"By Richard Bennett, Director of Amnesty International's Asia Pacific Program\nIn October last year, Htin Lin Oo, a writer and former information officer for Myanmar's main opposition party, the Na...",17 March 2015,News,https://www.amnesty.org/en/latest/news/2015/03/buddhist-nationalists-stoke-hatred-in-myanmar/,"<div class=""col__content"">\n<h1 class=""h2 heading--in-padded"" data-start-content="""">Buddhist nationalists stoke hatred in Myanmar</h1>\n<div class=""meta"">\n<div class=""meta__section"">\n<div>\n<tim...","By Richard Bennett, Director of Amnesty International's Asia Pacific Program\nIn October last year, Htin Lin Oo, a writer and former information officer for Myanmar's main opposition party, the Na...","[Asia and The Pacific, Myanmar, Discrimination, Religious Groups, Women's Rights, Women and Girls, Maternal Health and Reproductive Rights]",https://www.amnesty.org/en/articles/news/2015/03/myanmar-race-and-religion-laws/,,NaN,NaN
15312,https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date&p=958,"<ar

In [7]:
headers={
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36",
    'sec-ch-ua':'"Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"',
    'Referer': 'https://www.amnesty.org/',
    'sec-ch-ua-mobile': '?0'
}


def get_read_more(url):
    if url =='':
        return
    
    try:
        read_more_page = requests.get(url, headers= headers).content      
        read_more_soup = BeautifulSoup(read_more_page, 'html.parser')
        read_more_find = read_more_soup.find_all('div', class_='wysiwyg')
        read_more_text = read_more_find[0].text if len(read_more_find)>0 else ''
        read_more_text =  re.sub("^\s+|\s+$", "", read_more_text, flags=re.UNICODE) 
        return read_more_text
        
    except:
        print('exception', url)
        return

In [8]:
df2.loc[df2.read_more_link!='','read_more_link']

142      https://www.amnesty.org/en/latest/news/2017/09/saudi-arabia-wave-of-arrests-targets-last-vestiges-of-freedom-of-expression/
8337                                                                        https://www.amnesty.org/en/documents/asa38/5935/2017/en/
13309                                               https://www.amnesty.org/en/articles/news/2015/03/myanmar-race-and-religion-laws/
15312                    http://www.amnesty.org/en/news/azerbaijan-chair-council-europe-amid-climate-fear-and-suppression-2014-05-06
18261                                                     http://globalpublicsquare.blogs.cnn.com/2013/03/11/aleppos-river-of-death/
                                                                    ...                                                             
53346                                                            https://www.amnesty.org/en/countries/africa/liberia/report-liberia/
53347                                                                

In [9]:
df2.loc[df2.read_more_link!='','read_more_content'] =\
df2.loc[df2.read_more_link!='','read_more_link'].parallel_apply(get_read_more).values

In [12]:
df2

,url,articles,headline,abstract,date,content_type,link,html,full_story,topics,read_more_link,pdf_link,read_more_content,pdf_content
0,https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date&p=1,"<article class=""search-item--list"">\n<a class=""search-item__link search-item__link--list clearfix"" data-search-link="""" href=""/en/latest/research/2021/02/vietnam-phishing-attacks-research-vn/"">\n<d...",Nhấp chuột và Mồi nhử: Các nhà bảo vệ nhân quyền Việt Nam bị tấn công bằng phần mềm gián điệp,Một cuộc điều tra mới của Ân xá Quốc tế vừa xác định một chiến dịch tấn công bằng phần mềm gián điệp nhắm vào những người bảo vệ nhân quyền (HRDs) Việt Nam từ tháng 2 năm 2018 đến tháng 11 năm 202...,24 February 2021,Blog,https://www.amnesty.org/en/latest/research/2021/02/vietnam-phishing-attacks-research-vn/,"<div class=""col__content"">\n<h1 class=""h2 heading--in-padded"" data-start-content="""">Nhấp chuột và Mồi nhử: Các nhà bảo vệ nhân quyền Việt Nam bị tấn công bằng phần mềm gián điệp </h1>\n<div class=...",Một cuộc điều tra mới của Ân xá Quốc tế vừa xác định một chiến dịch tấn công bằng phần mềm gián điệp nhắm vào những người bảo vệ nhân quyền (HRDs) Việt Nam từ tháng 2 năm 2018 đến tháng 11 năm 202...,,,,NaN,NaN
1,https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date&p=1,"<article class=""search-item--list"">\n<a class=""search-item__link search-item__link--list clearfix"" data-search-link="""" href=""/en/latest/research/2021/02/vietnam-hacking-group-activists-vn-pr/"">\n<...",Các nhà hoạt động Việt Nam bị nhóm hacker khét tiếng nhắm mục tiêu,"Nhóm chuyên xâm nhập mạng Ocean Lotus, nhóm bị nghi ngờ có liên hệ với chính phủ Việt Nam, là tổ chức đứng đằng sau một chiến dịch tấn công kéo dài dùng phần mềm gián điệp nhắm vào các nhà hoạt độ...",24 February 2021,Article,https://www.amnesty.org/en/latest/research/2021/02/vietnam-hacking-group-activists-vn-pr/,"<div class=""col__content"">\n<h1 class=""heading--main heading--in-padded"" data-start-content="""">\r\n\t\tCác nhà hoạt động Việt Nam bị nhóm hacker khét tiếng nhắm mục tiêu \r\n\t</h1>\n<div class=""m...","Nhóm chuyên xâm nhập mạng Ocean Lotus, nhóm bị nghi ngờ có liên hệ với chính phủ Việt Nam, là tổ chức đứng đằng sau một chiến dịch tấn công kéo dài dùng phần mềm gián điệp nhắm vào các nhà hoạt độ...",,,,NaN,NaN
2,https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date&p=1,"<article class=""search-item--list"">\n<a class=""search-item__link search-item__link--list clearfix"" data-search-link="""" href=""/en/latest/news/2021/02/viet-nam-hacking-group-targets-activist/"">\n<di...",Vietnamese activists targeted by notorious hacking group,"Hacking group Ocean Lotus, which has been suspected of having links with the Vietnamese government, is behind a sustained campaign of spyware attacks on the country’s human rights activists, a new...",24 February 2021,News,https://www.amnesty.org/en/latest/news/2021/02/viet-nam-hacking-group-targets-activist/,"<div class=""col__content"">\n<h1 class=""heading--main heading--in-padded"" data-start-content="""">\r\n\t\tVietnamese activists targeted by notorious hacking group\r\n\t</h1>\n<div class=""meta"">\n<div...","Hacking group Ocean Lotus, which has been suspected of having links with the Vietnamese government, is behind a sustained campaign of spyware attacks on the country’s human rights activists, a new...","[Viet Nam, Human Rights Defenders and Activists, Technology and Human Rights]",,,NaN,NaN
3,https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date&p=1,"<article class=""search-item--list"">\n<a class=""search-item__link search-item__link--list clearfix"" data-search-link="""" href=""/en/latest/news/2021/02/pakistan-christian-couple-death-row/"">\n<div cl...",Pakistan: Christian couple on death row for ‘blasphemous texts’ must be released,"The Pakistani authorities must immediately and unconditionally release a husband and wife facing death sentences after being conv

In [14]:
df2.to_csv('amnesty_read_more.csv',index=False)

# 5.  get external PDF files

In [3]:
df2= pd.read_csv('amnesty_read_more.csv',header= [0])

In [3]:
headers = {
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'zh,zh-TW;q=0.9,zh-CN;q=0.8,en;q=0.7',
# 'Cache-Control': 'max-age=0',
'Connection': 'keep-alive',
'Host': 'www.amnesty.org',
'sec-ch-ua': '"Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"',
'sec-ch-ua-mobile': '?0',
'Sec-Fetch-Dest': 'document',
'Sec-Fetch-Mode': 'navigate',
# 'Sec-Fetch-Site': 'same-origin',
'Sec-Fetch-User': '?1',
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36'
}
headers = {}


def download_pdf(url):
    
    if url =='':
        return ''
    
    filename = url.split('/')[-1]
    file = './pdf/%s'%filename

    if not os.path.exists(file):
        try:
            response = requests.get(url, headers = headers)
            my_raw_data = response.content
            with open(file, 'wb') as my_data:
                my_data.write(my_raw_data)
            return file
        except:
            print('exception: can\'t open the link', url)
            return ''
    else:
        return file

def pdf2text(path):
            
    full_text = ''
    try:         
        with open(file, 'rb') as open_pdf_file:
            read_pdf = PyPDF2.PdfFileReader(open_pdf_file)

            for page in range(read_pdf.numPages): 
                pageObj = read_pdf.getPage(page) 
                text = pageObj.extractText()
                full_text = full_text + '\n\n'+ text
                
    except:
        print('invalid pdf file', path)

    return full_text

In [4]:
pandarallel.initialize(nb_workers=64)

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
df2.loc[df2.pdf_link.isnull(),'pdf_link'] = ''
df2[['pdf_path']]=''
df2.head(10)

,url,articles,headline,abstract,date,content_type,link,html,full_story,topics,read_more_link,pdf_link,read_more_content,pdf_content,pdf_path
0,https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date&p=1,"<article class=""search-item--list"">\n<a class=""search-item__link search-item__link--list clearfix"" data-search-link="""" href=""/en/latest/research/2021/02/vietnam-phishing-attacks-research-vn/"">\n<d...",Nhấp chuột và Mồi nhử: Các nhà bảo vệ nhân quyền Việt Nam bị tấn công bằng phần mềm gián điệp,Một cuộc điều tra mới của Ân xá Quốc tế vừa xác định một chiến dịch tấn công bằng phần mềm gián điệp nhắm vào những người bảo vệ nhân quyền (HRDs) Việt Nam từ tháng 2 năm 2018 đến tháng 11 năm 202...,24 February 2021,Blog,https://www.amnesty.org/en/latest/research/2021/02/vietnam-phishing-attacks-research-vn/,"<div class=""col__content"">\n<h1 class=""h2 heading--in-padded"" data-start-content="""">Nhấp chuột và Mồi nhử: Các nhà bảo vệ nhân quyền Việt Nam bị tấn công bằng phần mềm gián điệp </h1>\n<div class=...",Một cuộc điều tra mới của Ân xá Quốc tế vừa xác định một chiến dịch tấn công bằng phần mềm gián điệp nhắm vào những người bảo vệ nhân quyền (HRDs) Việt Nam từ tháng 2 năm 2018 đến tháng 11 năm 202...,NaN,NaN,,NaN,NaN,
1,https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date&p=1,"<article class=""search-item--list"">\n<a class=""search-item__link search-item__link--list clearfix"" data-search-link="""" href=""/en/latest/research/2021/02/vietnam-hacking-group-activists-vn-pr/"">\n<...",Các nhà hoạt động Việt Nam bị nhóm hacker khét tiếng nhắm mục tiêu,"Nhóm chuyên xâm nhập mạng Ocean Lotus, nhóm bị nghi ngờ có liên hệ với chính phủ Việt Nam, là tổ chức đứng đằng sau một chiến dịch tấn công kéo dài dùng phần mềm gián điệp nhắm vào các nhà hoạt độ...",24 February 2021,Article,https://www.amnesty.org/en/latest/research/2021/02/vietnam-hacking-group-activists-vn-pr/,"<div class=""col__content"">\n<h1 class=""heading--main heading--in-padded"" data-start-content="""">\r\n\t\tCác nhà hoạt động Việt Nam bị nhóm hacker khét tiếng nhắm mục tiêu \r\n\t</h1>\n<div class=""m...","Nhóm chuyên xâm nhập mạng Ocean Lotus, nhóm bị nghi ngờ có liên hệ với chính phủ Việt Nam, là tổ chức đứng đằng sau một chiến dịch tấn công kéo dài dùng phần mềm gián điệp nhắm vào các nhà hoạt độ...",NaN,NaN,,NaN,NaN,
2,https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date&p=1,"<article class=""search-item--list"">\n<a class=""search-item__link search-item__link--list clearfix"" data-search-link="""" href=""/en/latest/news/2021/02/viet-nam-hacking-group-targets-activist/"">\n<di...",Vietnamese activists targeted by notorious hacking group,"Hacking group Ocean Lotus, which has been suspected of having links with the Vietnamese government, is behind a sustained campaign of spyware attacks on the country’s human rights activists, a new...",24 February 2021,News,https://www.amnesty.org/en/latest/news/2021/02/viet-nam-hacking-group-targets-activist/,"<div class=""col__content"">\n<h1 class=""heading--main heading--in-padded"" data-start-content="""">\r\n\t\tVietnamese activists targeted by notorious hacking group\r\n\t</h1>\n<div class=""meta"">\n<div...","Hacking group Ocean Lotus, which has been suspected of having links with the Vietnamese government, is behind a sustained campaign of spyware attacks on the country’s human rights activists, a new...","['Viet Nam', 'Human Rights Defenders and Activists', 'Technology and Human Rights']",NaN,,NaN,NaN,
3,https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date&p=1,"<article class=""search-item--list"">\n<a class=""search-item__link search-item__link--list clearfix"" data-search-link="""" href=""/en/latest/news/2021/02/pakistan-christian-couple-death-row/"">\n<div cl...",Pakistan: Christian couple on death row for ‘blasphemous texts’ must be released,"The Pakistani authorities must immediately and unconditionally release a husband and wife facing

In [6]:
batch_size = 512
for i in range(0,df2.shape[0],batch_size):
    print(i)
    df2.loc[i:i+batch_size-1,'pdf_path'] = df2.loc[i:i+batch_size-1,'pdf_link'].parallel_apply(download_pdf).values

0
512
1024
1536
2048
2560
3072
3584
4096
4608
5120
5632
6144
6656
7168
7680
8192
8704
9216
9728
10240
10752
11264
11776
12288
12800
13312
13824
14336
14848
15360
15872
16384
16896
17408
17920
18432
18944
19456
19968
20480
20992
21504
22016
22528
23040
23552
24064
24576
25088
25600
26112
26624
27136
27648
28160
28672
29184
29696
30208
30720
31232
31744
32256
32768
33280
33792
34304
34816
35328
35840
36352
36864
37376
37888
38400
38912
39424
39936
40448
40960
41472
41984
42496
43008
43520
44032
44544
45056
45568
46080
46592
47104
47616
48128
48640
49152
49664
50176
50688
51200
51712
52224
52736
53248


In [8]:
df2.to_csv('amnesty_pdf_path.csv', index =False)

In [27]:
df3 = df2[['pdf_link','pdf_path']].dropna(subset=['pdf_link']).reset_index(drop=True)
df3.to_csv('amnesty_only_pdf.csv', index =False)

# 6. Convert pdf to txt
we only modify pdf dataframe to save memory

In [2]:
df3 = pd.read_csv('amnesty_only_pdf.csv', header=[0])
df3

,pdf_link,pdf_path
0,https://www.amnesty.org/download/Documents/ASA3537442021ENGLISH.pdf,./pdf/ASA3537442021ENGLISH.pdf
1,https://www.amnesty.org/download/Documents/EUR4437422021ENGLISH.pdf,./pdf/EUR4437422021ENGLISH.pdf
2,https://www.amnesty.org/download/Documents/IOR4037372021ENGLISH.PDF,./pdf/IOR4037372021ENGLISH.PDF
3,https://www.amnesty.org/download/Documents/MDE1337432021ENGLISH.pdf,./pdf/MDE1337432021ENGLISH.pdf
4,https://www.amnesty.org/download/Documents/MDE2837072021ENGLISH.pdf,./pdf/MDE2837072021ENGLISH.pdf
...,...,...
41644,https://www.amnesty.org/download/Documents/204000/NWS210031961eng.pdf,./pdf/NWS210031961eng.pdf
41645,https://www.amnesty.org/download/Documents/204000/nws210021961en.pdf,./pdf/nws210021961en.pdf
41646,https://www.amnesty.org/download/Documents/NWS210011961ENGLISH.PDF,./pdf/NWS210011961ENGLISH.PDF
41647,https://www.amnesty.org/download/Documents/204000/act100011961en.pdf,./pdf/act100011961en.pdf


In [14]:
def pdf2txt(pdf_path, save_file = False):
    txt_path = './txt/{}.txt'.format(pdf_path.split('/')[-1][:-4])
    
    try:           
        text = pdfminer.high_level.extract_text(pdf_path)
        
        if save_file:
            with open(txt_path, 'w') as open_txt_file:
                print(text, file = open_txt_file)
                
        return pd.Series([txt_path, text])
    
    except:
        print('invalid pdf file', pdf_path)
        return pd.Series(['', ''])

In [15]:
pandarallel.initialize()
df3[['txt_path', 'txt']] = df3['pdf_path'].parallel_apply(pdf2txt,save_file = False).values

INFO: Pandarallel will run on 36 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
invalid pdf file ./pdf/afr460442002en.pdf
invalid pdf file ./pdf/afr460442002en.pdf
invalid pdf file ./pdf/amr460052007.pdf
invalid pdf file ./pdf/amr250072011en.pdf
invalid pdf file ./pdf/amr511302008en.pdf
invalid pdf file ./pdf/ASA2840122016ENGLISH.pdf
invalid pdf file ./pdf/EUR6217422015.pdf
invalid pdf file ./pdf/AFR4451312016ENGLISH.pdf
invalid pdf file ./pdf/eur210042009eng.pdf
invalid pdf file ./pdf/AFR6231832016ENGLISH.PDF
invalid pdf file ./pdf/eur440212013en.pdf
invalid pdf file ./pdf/ASA1625772020ENGLISH.PDF
invalid pdf file ./pdf/EUR4651472016ENGLISH.PDF
invalid pdf file ./pdf/mde141762003en.pdf
invalid pdf file ./pdf/amr510282012en.pdf
invalid pdf file ./pdf/ASA4197882019ENGLISH.PDF
invalid pdf file ./pdf/asa170492000en.pdf
invalid pdf file ./pdf/mde290012010en.pdf
invalid pdf file ./pdf/asa210172011en.pdf
invalid pdf file ./pdf/IO

Process ForkPoolWorker-70:
Process ForkPoolWorker-67:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-69:
Traceback (most recent call last):
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-53:


KeyboardInterrupt: 

Process ForkPoolWorker-60:
Process ForkPoolWorker-72:
Process ForkPoolWorker-55:
Process ForkPoolWorker-73:
Process ForkPoolWorker-66:
Process ForkPoolWorker-68:
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-62:
Process ForkPoolWorker-71:
Process ForkPoolWorker-65:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-64:
Traceback (most recent call last):
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._

invalid pdf file ./pdf/amr460052007.pdf


invalid pdf file ./pdf/eur460021972.pdf


In [4]:
df3.to_csv('amnesty_only_pdf_n_text.csv', index =False)

NameError: name 'df3' is not defined

# 4. Further process

In [8]:
# df1 = pd.read_csv('amnesty_extracted_articles.csv',header = [0])
# df1.full_story.to_csv('amnesty_only_text.csv',index= False)

# df = pd.read_csv('amnesty_read_more.csv',header = [0])
# df[['full_story', 'read_more_content']].to_csv('amnesty_n_read_more_only_text.csv',index= False)

In [8]:
df = pd.read_csv('amnesty_read_more.csv',header = [0])
df = df.iloc[2:]
df

,url,articles,headline,abstract,date,content_type,link,html,full_story,topics,read_more_link,pdf_link,read_more_content,pdf_content
2,https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date&p=1,"<article class=""search-item--list"">\n<a class=""search-item__link search-item__link--list clearfix"" data-search-link="""" href=""/en/latest/news/2021/02/viet-nam-hacking-group-targets-activist/"">\n<div class=""search-item__content search-item__content--list"">\n<div class="""">\n<ul class=""tags"">\n<li class=""tags__item"">\n<span class=""tags__icon--news"">\r\n\t\t\t\t\t\t\tNews\r\n\t\t\t\t\t\t</span>\n</...",Vietnamese activists targeted by notorious hacking group,"Hacking group Ocean Lotus, which has been suspected of having links with the Vietnamese government, is behind a sustained campaign of spyware attacks on the country’s human rights activists, a new investigation by Amnesty Tech reveals, underscoring an intensifying assault on freedom of expression. Amnesty Tech’s Security Lab found technical evidence in phishing emails sent to two prominent Vie...",24 February 2021,News,https://www.amnesty.org/en/latest/news/2021/02/viet-nam-hacking-group-targets-activist/,"<div class=""col__content"">\n<h1 class=""heading--main heading--in-padded"" data-start-content="""">\r\n\t\tVietnamese activists targeted by notorious hacking group\r\n\t</h1>\n<div class=""meta"">\n<div class=""meta__section"">\n<div>\n<time datetime=""2021-02-24T15:30:09"">24 February 2021, 15:30 UTC</time>\n</div>\n</div>\n</div>\n<div class=""wysiwyg"">\n<p>Hacking group Ocean Lotus, which has been sus...","Hacking group Ocean Lotus, which has been suspected of having links with the Vietnamese government, is behind a sustained campaign of spyware attacks on the country’s human rights activists, a new investigation by Amnesty Tech reveals, underscoring an intensifying assault on freedom of expression.\n\n\r\n\t\tThese latest attacks by Ocean Lotus highlight the repression Vietnamese activists at h...","['Viet Nam', 'Human Rights Defenders and Activists', 'Technology and Human Rights']",NaN,NaN,NaN,NaN
3,https://www.amnesty.org/en/search/?q=&ref=&year=&lang=en&adv=1&sort=date&p=1,"<article class=""search-item--list"">\n<a class=""search-item__link search-item__link--list clearfix"" data-search-link="""" href=""/en/latest/news/2021/02/pakistan-christian-couple-death-row/"">\n<div class=""search-item__content search-item__content--list"">\n<div class="""">\n<ul class=""tags"">\n<li class=""tags__item"">\n<span class=""tags__icon--news"">\r\n\t\t\t\t\t\t\tArticle\r\n\t\t\t\t\t\t</span>\n</l...",Pakistan: Christian couple on death row for ‘blasphemous texts’ must be released,"The Pakistani authorities must immediately and unconditionally release a husband and wife facing death sentences after being convicted of sending ‘blasphemous’ text messages, said Amnesty International, ahead of an appeal hearing in the case at the Lahore High Court today. Shagufta Kausar and Shafqat Emmanuel, who are Christians, have been in prison since 2013 and were convicted and sentenced ...",24 February 2021,Article,https://www.amnesty.org/en/latest/news/2021/02/pakistan-christian-couple-death-row/,"<div class=""col__content"">\n<h1 class=""heading--main heading--in-padded"" data-start-content="""">\r\n\t\tPakistan: Christian couple on death row for ‘blasphemous texts’ must be released \r\n\t</h1>\n<div class=""meta"">\n<div class=""meta__section"">\n<div>\n<time datetime=""2021-02-24T13:40:37"">24 February 2021, 13:40 UTC</time>\n</div>\n</div>\n</div>\n<div class=""wysiwyg"">\n<p><span>The Pakistani ...","The Pakistani authorities must immediately and unconditionally release a husband and wife facing death sentences after being convicted of sending ‘blasphemous’ text messages, said Amnesty International, ahead of an appeal hearing in the case at the Lahore High Court today. \nShagufta Kausar and Shafqat Emmanuel, who are Christians, have been in prison since 2013 and wer

In [10]:
df =df.iloc[:,[6,4,2,3,5,8,9,10,12,11,13]].reset_index(drop=True)
df.to_csv('df_temp.csv', index=False)

In [36]:
df1 = pd.read_csv('df_temp.csv',header=0)
df1

,link,date,headline,abstract,content_type,full_story,topics,read_more_link,read_more_content,pdf_link,pdf_content
0,https://www.amnesty.org/en/latest/news/2021/02/viet-nam-hacking-group-targets-activist/,24 February 2021,Vietnamese activists targeted by notorious hacking group,"Hacking group Ocean Lotus, which has been suspected of having links with the Vietnamese government, is behind a sustained campaign of spyware attacks on the country’s human rights activists, a new investigation by Amnesty Tech reveals, underscoring an intensifying assault on freedom of expression. Amnesty Tech’s Security Lab found technical evidence in phishing emails sent to two prominent Vie...",News,"Hacking group Ocean Lotus, which has been suspected of having links with the Vietnamese government, is behind a sustained campaign of spyware attacks on the country’s human rights activists, a new investigation by Amnesty Tech reveals, underscoring an intensifying assault on freedom of expression.\n\n\r\n\t\tThese latest attacks by Ocean Lotus highlight the repression Vietnamese activists at h...","['Viet Nam', 'Human Rights Defenders and Activists', 'Technology and Human Rights']",NaN,NaN,NaN,NaN
1,https://www.amnesty.org/en/latest/news/2021/02/pakistan-christian-couple-death-row/,24 February 2021,Pakistan: Christian couple on death row for ‘blasphemous texts’ must be released,"The Pakistani authorities must immediately and unconditionally release a husband and wife facing death sentences after being convicted of sending ‘blasphemous’ text messages, said Amnesty International, ahead of an appeal hearing in the case at the Lahore High Court today. Shagufta Kausar and Shafqat Emmanuel, who are Christians, have been in prison since 2013 and were convicted and sentenced ...",Article,"The Pakistani authorities must immediately and unconditionally release a husband and wife facing death sentences after being convicted of sending ‘blasphemous’ text messages, said Amnesty International, ahead of an appeal hearing in the case at the Lahore High Court today. \nShagufta Kausar and Shafqat Emmanuel, who are Christians, have been in prison since 2013 and were convicted and sentenc...",NaN,NaN,NaN,NaN,NaN
2,https://www.amnesty.org/en/latest/research/2021/02/click-and-bait-vietnamese-human-rights-defenders-targeted-with-spyware-attacks/,24 February 2021,Click and Bait: Vietnamese Human Rights Defenders Targeted with Spyware Attacks,"A new Amnesty International investigation has identified a campaign of spyware attacks targeting Vietnamese human rights defenders (HRDs) from February 2018 to November 2020. Amnesty International’s Security Lab attributes these attacks to an attack group known as Ocean Lotus. The group has been active since at least 2014, targeting the private sector and HRDs. The spyware attacks investigated...",Blog,"A new Amnesty International investigation has identified a campaign of spyware attacks targeting Vietnamese human rights defenders (HRDs) from February 2018 to November 2020. Amnesty International’s Security Lab attributes these attacks to an attack group known as Ocean Lotus. The group has been active since at least 2014, targeting the private sector and HRDs. \nThe spyware attacks investigat...","['Viet Nam', 'Technology and Human Rights']",NaN,NaN,NaN,NaN
3,https://www.amnesty.org/en/documents/asa35/3744/2021/en/,24 February 2021,Philippines: Detained Senator's prolonged detention a grave injustice,"Amnesty International marks with grave concern the ongoing arbitrary detention of human rights defender and Filipino Senator Leila de Lima, now reaching its fourth year on 24 February 2021. Amidst the worsening human rights situation in the country, the organization continues to strongly urge the Philippine government to drop the politically motivated charges against her, order her immediate a...",Research,"Amnesty International marks with grave concern the ongoing arbitrary detention of human rights defender and Filipino Senator Leila de Lim

In [37]:
def remove_noise(x):
    find_start = re.findall('/\*\! \r\n \* Base CSS for pdf2htmlEX',x)
    find_end = re.findall('\w\d+{\w+:-?\d+\.\d+px;}',x)
    try:
        start = find_start[0]
        end = find_end[len(find_end)-1]
        x = x.split(start)[0]+ '\n' + x.split(end)[1]
    except:
        print('None')
    return x

In [38]:
pandarallel.initialize(nb_workers=64, progress_bar=True)

df1.loc[df1.full_story.str.contains('pdf2htmlEX')==True, 'full_story']=\
df1.loc[df1.full_story.str.contains('pdf2htmlEX')==True, 'full_story'].parallel_apply(remove_noise).values

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [32]:
# df1.full_story = df1.full_story + '\n' + df1.read_more_content

In [33]:
# df1[~df1.text.isnull()]

AttributeError: 'DataFrame' object has no attribute 'text'

In [39]:
df1 = df1[['link','date', 'headline','abstract','content_type','full_story','topics','read_more_content','read_more_link', 'pdf_link']]
df1.columns=['url','date', 'title','abstract','types','text','topic','read_more', 'read_more_link','pdf_link']
df1

,url,date,title,abstract,types,text,topic,read_more,read_more_link,pdf_link
0,https://www.amnesty.org/en/latest/news/2021/02/viet-nam-hacking-group-targets-activist/,24 February 2021,Vietnamese activists targeted by notorious hacking group,"Hacking group Ocean Lotus, which has been suspected of having links with the Vietnamese government, is behind a sustained campaign of spyware attacks on the country’s human rights activists, a new investigation by Amnesty Tech reveals, underscoring an intensifying assault on freedom of expression. Amnesty Tech’s Security Lab found technical evidence in phishing emails sent to two prominent Vie...",News,"Hacking group Ocean Lotus, which has been suspected of having links with the Vietnamese government, is behind a sustained campaign of spyware attacks on the country’s human rights activists, a new investigation by Amnesty Tech reveals, underscoring an intensifying assault on freedom of expression.\n\n\r\n\t\tThese latest attacks by Ocean Lotus highlight the repression Vietnamese activists at h...","['Viet Nam', 'Human Rights Defenders and Activists', 'Technology and Human Rights']",NaN,NaN,NaN
1,https://www.amnesty.org/en/latest/news/2021/02/pakistan-christian-couple-death-row/,24 February 2021,Pakistan: Christian couple on death row for ‘blasphemous texts’ must be released,"The Pakistani authorities must immediately and unconditionally release a husband and wife facing death sentences after being convicted of sending ‘blasphemous’ text messages, said Amnesty International, ahead of an appeal hearing in the case at the Lahore High Court today. Shagufta Kausar and Shafqat Emmanuel, who are Christians, have been in prison since 2013 and were convicted and sentenced ...",Article,"The Pakistani authorities must immediately and unconditionally release a husband and wife facing death sentences after being convicted of sending ‘blasphemous’ text messages, said Amnesty International, ahead of an appeal hearing in the case at the Lahore High Court today. \nShagufta Kausar and Shafqat Emmanuel, who are Christians, have been in prison since 2013 and were convicted and sentenc...",NaN,NaN,NaN,NaN
2,https://www.amnesty.org/en/latest/research/2021/02/click-and-bait-vietnamese-human-rights-defenders-targeted-with-spyware-attacks/,24 February 2021,Click and Bait: Vietnamese Human Rights Defenders Targeted with Spyware Attacks,"A new Amnesty International investigation has identified a campaign of spyware attacks targeting Vietnamese human rights defenders (HRDs) from February 2018 to November 2020. Amnesty International’s Security Lab attributes these attacks to an attack group known as Ocean Lotus. The group has been active since at least 2014, targeting the private sector and HRDs. The spyware attacks investigated...",Blog,"A new Amnesty International investigation has identified a campaign of spyware attacks targeting Vietnamese human rights defenders (HRDs) from February 2018 to November 2020. Amnesty International’s Security Lab attributes these attacks to an attack group known as Ocean Lotus. The group has been active since at least 2014, targeting the private sector and HRDs. \nThe spyware attacks investigat...","['Viet Nam', 'Technology and Human Rights']",NaN,NaN,NaN
3,https://www.amnesty.org/en/documents/asa35/3744/2021/en/,24 February 2021,Philippines: Detained Senator's prolonged detention a grave injustice,"Amnesty International marks with grave concern the ongoing arbitrary detention of human rights defender and Filipino Senator Leila de Lima, now reaching its fourth year on 24 February 2021. Amidst the worsening human rights situation in the country, the organization continues to strongly urge the Philippine government to drop the politically motivated charges against her, order her immediate a...",Research,"Amnesty International marks with grave concern the ongoing arbitrary detention of human rights defender and Filipino Senator Leila de Lima, now reaching its fourth year on 24 February 202

In [42]:
df1 = df1[~df1.duplicated()]
df1= df1.sort_values('date').reset_index(drop=True)
df1

,url,date,title,abstract,types,text,topic,read_more,read_more_link,pdf_link
0,https://www.amnesty.org/en/documents/nws21/015/1962/en/,1 April 1962,"Amnesty Journal of the Amnesty movement, Number 1, 1962",The first Amnesty Quarterly of 1962 contains a series of reports on the practical work that Amnesty has done this year.,Research,The first Amnesty Quarterly of 1962 contains a series of reports on the practical work that Amnesty has done this year.,NaN,NaN,NaN,https://www.amnesty.org/download/Documents/204000/nws210151962en.pdf
1,https://www.amnesty.org/en/documents/nws21/011/1962/en/,1 April 1962,"Amnesty Bulletin to national sections from London No. 1, April 1962",NaN,Research,NaN,NaN,NaN,NaN,https://www.amnesty.org/download/Documents/204000/nws210111962en.pdf
2,https://www.amnesty.org/en/documents/nws21/016/1964/en/,1 April 1964,"Eustomy: Quarterly Journal of Amnesty International, No.2, April 1964",NaN,Research,NaN,NaN,NaN,NaN,https://www.amnesty.org/download/Documents/204000/nws210161964en.pdf
3,https://www.amnesty.org/en/documents/nws21/015/1964/en/,1 April 1964,"Amnesty International Bulletin (British Section) No.7, April 1964",NaN,Research,NaN,NaN,NaN,NaN,https://www.amnesty.org/download/Documents/204000/nws210151964en.pdf
4,https://www.amnesty.org/en/documents/nws21/002/1965/en/,1 April 1965,"Eustomy: Quarterly Journal of Amnesty International, No.6, April 1965",NaN,Research,NaN,NaN,NaN,NaN,https://www.amnesty.org/download/Documents/204000/nws210021965en.pdf
...,...,...,...,...,...,...,...,...,...,...
52947,https://www.amnesty.org/en/countries/africa/liberia/,NaN,Liberia,"Domestic violence, and sexual violence against women and girls remained widespread. Impunity for human rights violations persisted. Prison conditions did not meet international standards and individuals were frequently held in prolonged pre-trial detention. Read more.",Country profile,NaN,NaN,"Domestic violence, and sexual violence against women and girls remained widespread. Impunity for human rights violations persisted. Prison conditions did not meet international standards and individuals were frequently held in prolonged pre-trial detention. \n\n Background\n \nPresidential and legislative elections began in October. George Weah, of the Congress for Democratic Change party,...",https://www.amnesty.org/en/countries/africa/liberia/report-liberia/,NaN
52948,https://www.amnesty.org/en/countries/africa/togo/,NaN,Togo,"The authorities continued to crackdown on peaceful dissent, particularly against members of the opposition Pan-African National Party ahead of the 2020 presidential election. Togo adopted several laws curtailing the rights to freedom of expression and peaceful assembly. Excessive use of force, arbitrary arrests and detentions, torture and other ill-treatment, and impunity for human rights viol...",Country profile,NaN,NaN,"The authorities continued to crackdown on peaceful dissent, particularly against members of the opposition Pan-African National Party ahead of the 2020 presidential election. Togo adopted several laws curtailing the rights to freedom of expression and peaceful assembly. Excessive use of force, arbitrary arrests and detentions, torture and other ill-treatment, and impunity for human rights viol...",https://www.amnesty.org/en/countries/africa/togo/report-togo/,NaN
52949,https://www.amnesty.org/en/countries/africa/somalia/,NaN,Somalia,"The ongoing conflict had a devasting impact on the civilian population and over 1,000 were killed and injured. The authorities cracked down on critics, including journalists, and opposition members stifling their rights to freedom of expression and assembly, in some cases using excessive force resulting in deaths. Al-Shabaab also severely restricted journalists’ ability to work. Sexual violenc...",Country profile,NaN,NaN,"The ongoing conflict had a devasting impact on the civilian population and over 1,000 were killed and injured. The authorities cracked down on critics, including journali

In [52]:
df1 = df1[~df1.duplicated(subset= ['url'])].reset_index(drop=True)
df1

,url,date,title,abstract,types,text,topic,read_more,read_more_link,pdf_link
0,https://www.amnesty.org/en/documents/nws21/015/1962/en/,1 April 1962,"Amnesty Journal of the Amnesty movement, Number 1, 1962",The first Amnesty Quarterly of 1962 contains a series of reports on the practical work that Amnesty has done this year.,Research,The first Amnesty Quarterly of 1962 contains a series of reports on the practical work that Amnesty has done this year.,NaN,NaN,NaN,https://www.amnesty.org/download/Documents/204000/nws210151962en.pdf
1,https://www.amnesty.org/en/documents/nws21/011/1962/en/,1 April 1962,"Amnesty Bulletin to national sections from London No. 1, April 1962",NaN,Research,NaN,NaN,NaN,NaN,https://www.amnesty.org/download/Documents/204000/nws210111962en.pdf
2,https://www.amnesty.org/en/documents/nws21/016/1964/en/,1 April 1964,"Eustomy: Quarterly Journal of Amnesty International, No.2, April 1964",NaN,Research,NaN,NaN,NaN,NaN,https://www.amnesty.org/download/Documents/204000/nws210161964en.pdf
3,https://www.amnesty.org/en/documents/nws21/015/1964/en/,1 April 1964,"Amnesty International Bulletin (British Section) No.7, April 1964",NaN,Research,NaN,NaN,NaN,NaN,https://www.amnesty.org/download/Documents/204000/nws210151964en.pdf
4,https://www.amnesty.org/en/documents/nws21/002/1965/en/,1 April 1965,"Eustomy: Quarterly Journal of Amnesty International, No.6, April 1965",NaN,Research,NaN,NaN,NaN,NaN,https://www.amnesty.org/download/Documents/204000/nws210021965en.pdf
...,...,...,...,...,...,...,...,...,...,...
52928,https://www.amnesty.org/en/countries/africa/liberia/,NaN,Liberia,"Domestic violence, and sexual violence against women and girls remained widespread. Impunity for human rights violations persisted. Prison conditions did not meet international standards and individuals were frequently held in prolonged pre-trial detention. Read more.",Country profile,NaN,NaN,"Domestic violence, and sexual violence against women and girls remained widespread. Impunity for human rights violations persisted. Prison conditions did not meet international standards and individuals were frequently held in prolonged pre-trial detention. \n\n Background\n \nPresidential and legislative elections began in October. George Weah, of the Congress for Democratic Change party,...",https://www.amnesty.org/en/countries/africa/liberia/report-liberia/,NaN
52929,https://www.amnesty.org/en/countries/africa/togo/,NaN,Togo,"The authorities continued to crackdown on peaceful dissent, particularly against members of the opposition Pan-African National Party ahead of the 2020 presidential election. Togo adopted several laws curtailing the rights to freedom of expression and peaceful assembly. Excessive use of force, arbitrary arrests and detentions, torture and other ill-treatment, and impunity for human rights viol...",Country profile,NaN,NaN,"The authorities continued to crackdown on peaceful dissent, particularly against members of the opposition Pan-African National Party ahead of the 2020 presidential election. Togo adopted several laws curtailing the rights to freedom of expression and peaceful assembly. Excessive use of force, arbitrary arrests and detentions, torture and other ill-treatment, and impunity for human rights viol...",https://www.amnesty.org/en/countries/africa/togo/report-togo/,NaN
52930,https://www.amnesty.org/en/countries/africa/somalia/,NaN,Somalia,"The ongoing conflict had a devasting impact on the civilian population and over 1,000 were killed and injured. The authorities cracked down on critics, including journalists, and opposition members stifling their rights to freedom of expression and assembly, in some cases using excessive force resulting in deaths. Al-Shabaab also severely restricted journalists’ ability to work. Sexual violenc...",Country profile,NaN,NaN,"The ongoing conflict had a devasting impact on the civilian population and over 1,000 were killed and injured. The authorities cracked down on critics, including journali

In [57]:
df1.to_csv('df1.csv', index=False)